In [1]:
import pandas as pd

fh = "../files/picklist/candidates-sierra-export-clean-20240915.csv"

In [4]:
df = pd.read_csv(
    fh,
    delimiter="\t",
    encoding="utf-8",
    engine="python",
    keep_default_na=False,
    quotechar=None,
    quoting=3,
    names=[
        "iNumber", "iCode2", "iType", "iLocation", "iStatus", "iMessage", "iOpacMessage",
        "iAgency", "iTotalCheckouts", "iBarcode", "iUnit", "iCallNum", "iVolume", "iIntNote", "Message", "oversized",
        "linkednibsno", "DateType", "Date1", "Date2", "size", "title", "pubInfo", "bCallNum", "bStaffCallNum"],
    dtype={"iBarcode": str, "linkedbibsno": int}
    )


In [5]:
locs = sorted(df["iLocation"].unique())
locs_df = pd.DataFrame(dict(location=locs))
locs_df.to_csv("../files/picklist/locations.csv", index=False)

In [6]:
iTypes = sorted(df["iType"].unique())
iTypes_df = pd.DataFrame(dict(item_type=iTypes))
iTypes_df.to_csv("../files/picklist/item_types.csv", index=False)

In [7]:
def determine_location(row):
    if row["iLocation"].startswith("rc"):
        return "ReCAP"
    elif row["iLocation"].startswith("ma") and row["iLocation"][3] == "9":
        return "SASB-M2"
    elif row["iLocation"].startswith("ma") and row["iLocation"].endswith("82"):
        return "SASB-M1"
    elif row["iLocation"].startswith("ma"):
        return "SASB"
    elif row["iLocation"].startswith("sc"):
        return "SCH"
    elif row["iLocation"].startswith("pa"):
        return "LPA"
    elif row["iLocation"].startswith("bu"):
        return "SNFL"

In [9]:
def exclude(row):
    """
    Determines if an item will be excluded from the picklist. The state can be permanent 
    (item will be added to will-not-send list) or temporary (not included in the picklist
    nor the will-not-send list). Temp state items will have a chance to be digitized when
    their status changes, example: item is in preservation or is missing.
    """
    if not row["iLocation"].startswith(("bu", "ma", "pa", "rc", "sc", "ls")):
        return "TRUE-NOT-RL-LOC"
    
    # Schomburg specific requirements
    if row["iLocation"].startswith("scd"):
        # Schomburg rare books col
        return "TRUE-SCH-SCD"
    if row["iLocation"].startswith("sccc2"):
        # Schomburg Art and Artifacts Division
        return "TRUE-SCH-SCCC2"
    if row["iLocation"].startswith("scff1"):
        # Schomburg Open Shelf Collection
        return "TRUE-SCH-SCFF1"
    if row["iLocation"].startswith("scff3"):
        # Schomburg Reference Desk
        return "TRUE-SCH-SCFF3"
    if "schomburg collection" in row["iIntNote"].lower():
        # Arthur A. Schomburg Seed Collection
        return "TRUE-SCH-SEED"
    if row["iStatus"] == "k":
        # check w/staff per Schomburg's request
        return "TRUE-SCH-STATUS"
    if row["iOpacMessage"] == "4":
        # restricted use
        return "TRUE-SCH-RESTRICTED"
    if int(row["iType"]) > 101:
        return "TRUE-NOT-RL-ITYPE"

    if int(row["iType"]) in (
        6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 41, 42, 43, 50, 51, 52, 53, 61, 66, 68, 70, 72,
        73, 74, 75, 76, 79):
        return "TRUE-INCORRECT-ITYPE"
    if "google" in row["iIntNote"].lower():
        return "TRUE-PREV-PICK-REJECT"
    if "poor condition" in row["iIntNote"].lower():
        return "TRUE-POOR"
    if row["oversized"] == "False":
        return "TRUE-OVERSIZED"
    if len(row["iBarcode"]) != 14:
        return "TRUE-SHORT-BARCODE"
    if row["iBarcode"].isdigit() is False:
        return "TRUE-NON-NUMERIC-BARCODE"
    if row["iStatus"] in "$dlnqw":
        # items with permanent status preventing from being sent to Google
        return "TRUE-STATUS"
    if row["iStatus"] in "bcefimvxz":
        # items with temp status which may change next time
        return "FALSE-TEMP-BLOCK"
    # anything else deemed to be eligible
    return "FALSE"
    

In [10]:
df["location"] = df.apply(lambda row: determine_location(row), axis=1)

In [11]:
# total SASB items, info only
all_sasb_df = df[~(df["location"].isnull()) & (df["location"].str.startswith("SASB"))]

In [12]:
all_sasb_df.shape[0]

292226

In [13]:
df["will_not_send"] = df.apply(lambda row: exclude(row), axis=1)

In [14]:
all_sasb_without_ineligible_df = df[~(df["location"].isnull()) & (df["location"].str.startswith("SASB")) & (df["will_not_send"] == "FALSE")]
print(all_sasb_without_ineligible_df.shape[0])

220273


In [15]:
df_sasb = df[(df["location"] == "SASB") & (df["will_not_send"] == "FALSE")]
gdf_sasb = df_sasb.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "iVolume", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_sasb.to_csv("../files/picklist/sasb.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "iVolume", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [16]:
df_sasb.shape[0]

10710

In [17]:
df_m1 = df[(df["location"] == "SASB-M1") & (df["will_not_send"] == "FALSE")]
gdf_m1 = df_m1.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))  
gdf_m1.to_csv("../files/picklist/sasb-m1.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]) 

In [18]:
df_m1.shape[0]

30233

In [19]:
df_m2 = df[(df["location"] == "SASB-M2") & (df["will_not_send"] == "FALSE")]
df_m2.to_csv("../files/picklist/sasb-m2-detailed.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])
df_m2.to_csv("../files/picklist/sasb-m2-barcodes.csv", index=False, sep="\t", columns=["iBarcode"])

In [20]:
df_m2.shape[0]

179330

In [21]:
df_recap = df[(df["location"] == "ReCAP") & (df["will_not_send"] == "FALSE")]
df_recap.to_csv("../files/picklist/recap.csv", index=False, sep="\t", columns=["iBarcode"])

In [22]:
df_recap.shape[0]

850660

In [23]:
df_lpa = df[(df["location"] == "LPA") & (df["will_not_send"] == "FALSE")]
gdf_lpa = df_lpa.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_lpa.to_csv("../files/picklist/lpa.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [24]:
df_lpa.shape[0]

1065

In [25]:
sch_excluded_items = [
"i121579736",
"i172617042",
"i172617017",
"i176201002",
"i176201026",
"i176201014",
"i232441364",
"i232441376",
"i232441388",
"i241371831",
"i156828790",
"i156828789",
"i156828807",
"i156828819",
"i156828868",
"i156828844",
"i156828832",
"i156828820",
"i156828765",
"i156828753",
"i137759228",
"i232114675",
"i150129919",
"i159569461",
"i157004855",
"i147822610",
"i159404307",
"i150260258",
"i159448876",
"i147938107",
"i175606213",
"i340127156",
"i159523527",
"i159481107",
"i150286600",
"i103614175",
"i103655165",
"i103655542",
"i103913877",
"i103916842",
"i103933608",
"i103942178",
"i103952512",
"i103997763",
"i10403998x",
"i10404164x",
"i104091149",
"i10409168x",
"i104097711",
"i104215409",
"i104216542",
"i104226523",
"i104237752",
"i104258998",
"i104965083",
"i108143028",
"i10814303x",
"i111490157",
"i111490698",
"i111490704",
"i111490716",
"i111494345",
"i111497504",
"i111499896",
"i111499902",
"i111588297",
"i111599349",
"i112294558",
"i11229456x",
"i112294571",
"i112294583",
"i112294595",
"i112570823",
"i119269454",
"i121080110",
"i121356383",
"i121403348",
"i121411989",
"i121419137",
"i121547139",
"i12154848x",
"i121551350",
"i121599206",
"i121618316",
"i122088268",
"i12208827x",
"i122088281",
"i122088293",
"i12208830x",
"i122088311",
"i122088323",
"i122088335",
"i122088347",
"i122088359",
"i122631791",
"i122651881",
"i12340096x",
"i124137994",
"i127019534",
"i127020421",
"i127023392",
"i127024840",
"i127037299",
"i127037445",
"i127062105",
"i127071477",
"i127080259",
"i127117404",
"i127802733",
"i128498365",
"i132248451",
"i13243569x",
"i132442449",
"i132595734",
"i138041714",
"i138041726",
"i138041738",
"i138041751",
"i138041994",
"i138042019",
"i138042020",
"i138042044",
"i138042056",
"i138042068",
"i13804207x",
"i138042081",
"i138042093",
"i13804210x",
"i138042111",
"i138042123",
"i138042135",
"i138042147",
"i138042159",
"i138042172",
"i138042184",
"i138042196",
"i138042202",
"i138042214",
"i138042226",
"i138042238",
"i13804255x",
"i140765323",
"i140925120",
"i140925582",
"i140945015",
"i141153738",
"i14116086x",
"i141162892",
"i141192513",
"i141218472",
"i141221197",
"i141221239",
"i141256473",
"i141257593",
"i141260798",
"i141260804",
"i141334599",
"i141392228",
"i141833671",
"i144288643",
"i144288655",
"i144288667",
"i145208102",
"i148380372",
"i150021100",
"i150028660",
"i150029469",
"i150037740",
"i150123875",
"i150129415",
"i15013079x",
"i150132542",
"i150133042",
"i150135919",
"i150148562",
"i150149426",
"i150150040",
"i150153041",
"i150169516",
"i150169619",
"i150170300",
"i15017035x",
"i150171419",
"i15017164x",
"i150171663",
"i150171687",
"i150172278",
"i150172485",
"i150172655",
"i150173568",
"i150173672",
"i150173799",
"i150174652",
"i150174767",
"i150174810",
"i150175541",
"i150177318",
"i150178979",
"i150195102",
"i15019562x",
"i150196349",
"i150198401",
"i150198516",
"i150199697",
"i150199703",
"i150199752",
"i150199788",
"i150199971",
"i150201333",
"i150220789",
"i15022171x",
"i150221721",
"i150230515",
"i150233097",
"i150233103",
"i150233322",
"i150234338",
"i150234387",
"i150235574",
"i150235835",
"i150244137",
"i150244228",
"i15024423x",
"i150244241",
"i150244253",
"i150244265",
"i150244277",
"i150244289",
"i150244290",
"i150244307",
"i150244319",
"i15024454x",
"i150244551",
"i150254787",
"i150258057",
"i150263296",
"i150263302",
"i150263326",
"i150283581",
"i150285012",
"i150289431",
"i15029878x",
"i150299175",
"i150300906",
"i150301017",
"i150301686",
"i150309995",
"i150310468",
"i150313305",
"i150313342",
"i150313391",
"i15031341x",
"i150313433",
"i150313445",
"i150313950",
"i150324844",
"i150325216",
"i150325228",
"i15032523x",
"i150331915",
"i150768898",
"i150768904",
"i150768916",
"i150768928",
"i15076893x",
"i150810921",
"i150950111",
"i152137312",
"i152137324",
"i152137336",
"i152137348",
"i15213735x",
"i152137361",
"i152137373",
"i152137385",
"i152137397",
"i152139813",
"i152139825",
"i152139837",
"i152139849",
"i152139850",
"i152139862",
"i152139874",
"i152139886",
"i152139898",
"i152139904",
"i152139916",
"i152139928",
"i15213993x",
"i152139941",
"i152139953",
"i152139989",
"i152727723",
"i155128164",
"i155479076",
"i155479088",
"i15547909x",
"i155479106",
"i155479118",
"i15547912x",
"i155479131",
"i155479143",
"i155479155",
"i155479167",
"i155479179",
"i155479180",
"i155479192",
"i155479209",
"i155479210",
"i155479222",
"i159240517",
"i159354195",
"i159357019",
"i159359673",
"i159381472",
"i167389890",
"i240446173",
"i248323040",
"i260678259",
"i260678260",
"i409193239",
"i409193549",
"i40919394x",
"i409194165",
"i409194190",
"i409194475",
"i409195005",
"i409195601",
]

In [26]:
df_sch = df[(df["location"] == "SCH") & (df["will_not_send"] == "FALSE") & ~(df["iNumber"].isin(sch_excluded_items))]
gdf_sch = df_sch.groupby(["iLocation", "iType", "Date1"])[["iNumber", "iLocation", "iBarcode", "iType", "iCallNum", "iTotalCheckouts", "iStatus", "iMessage", "iOpacMessage", "title", "pubInfo", "Date1", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_sch.to_csv("../files/picklist/sch-review.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iType", "iCallNum", "iTotalCheckouts", "iStatus", "iMessage", "iOpacMessage", "title", "pubInfo", "Date1", "bCallNum", "bStaffCallNum"])

In [27]:
df_sch.shape[0]

16151

In [28]:
df_sch.to_csv("../files/picklist/sch-barcodes-only.csv", index=False, sep="\t", columns=["iBarcode"])

In [29]:
df_snfl = df[(df["location"] == "SNFL") & (df["will_not_send"] == "FALSE")]
gdf_snfl = df_snfl.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_snfl.to_csv("../files/picklist/snfl.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [30]:
df_snfl.shape[0]

18

In [31]:
df_will_not_send = df[(df["will_not_send"].str.startswith("TRUE")) | (df["iNumber"].isin(sch_excluded_items))]
df_will_not_send.to_csv("../files/picklist/will_not_send.csv", index=False, sep="\t", columns=["iBarcode"])

In [32]:
df_will_not_send.shape[0]

144890

In [33]:
df_sch[df_sch["iCallNum"] == ""].to_csv("../files/picklist/sch-missing-callnum.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [34]:
# invalid barcodes
df_invalid_barcodes = df[(df["iBarcode"].str.len() != 14)]
print(df_invalid_barcodes.shape[0])
df_invalid_barcodes.to_csv("../files/picklist/invalid_barcodes.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

42


In [35]:
# make sure the final count is correct
print(f"total # of items: {df.shape[0]}")
locs_total = df_sasb.shape[0] + df_m1.shape[0] + df_m2.shape[0] + df_recap.shape[0] + df_lpa.shape[0] + df_sch.shape[0] + df_snfl.shape[0] + df_will_not_send.shape[0]
print(f"sum of items in all locations: {locs_total}")

total # of items: 1233058
sum of items in all locations: 1233057


In [36]:
# Schomburg's excluded items
df_sch_excluded = df[(df["location"] == "SCH") & (df["will_not_send"].str.startswith("TRUE"))]
df_sch_excluded.shape[0]


2790

In [37]:
df_sch_excluded.to_csv("../files/picklist/sch-excluded.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "will_not_send"])

In [38]:
sch_reasons = df_sch_excluded["will_not_send"].value_counts()

In [39]:
sch_reasons.to_csv("../files/picklist/sch-excluded-reasons.csv", header=["count"], sep="\t")